imports:

In [110]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from collections import Counter
import random
#from graphviz import Digraph
from IPython.display import Image

load dataset:

In [111]:
data = pd.read_csv("heart_attack_prediction_dataset.csv")
data

,Patient ID,Age,Sex,Cholesterol,Blood Pressure,Heart Rate,Diabetes,Family History,Smoking,Obesity,...,Sedentary Hours Per Day,Income,BMI,Triglycerides,Physical Activity Days Per Week,Sleep Hours Per Day,Country,Continent,Hemisphere,Heart Attack Risk
0,BMW7812,67,Male,208,158/88,72,0,0,1,0,...,6.615001,261404,31.251233,286,0,6,Argentina,South America,Southern Hemisphere,0
1,CZE1114,21,Male,389,165/93,98,1,1,1,1,...,4.963459,285768,27.194973,235,1,7,Canada,North America,Northern Hemisphere,0
2,BNI9906,21,Female,324,174/99,72,1,0,0,0,...,9.463426,235282,28.176571,587,4,4,France,Europe,Northern Hemisphere,0
3,JLN3497,84,Male,383,163/100,73,1,1,1,0,...,7.648981,125640,36.464704,378,3,4,Canada,North America,Northern Hemisphere,0
4,GFO8847,66,Male,318,91/88,93,1,1,1,1,...,1.514821,160555,21.809144,231,1,5,Thailand,Asia,Northern Hemisphere,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8758,MSV9918,60,Male,121,94/76,61,1,1,1,0,...,10.806373,235420,19.655895,67,7,7,Thailand,Asia,Northern Hemisphere,0
8759,QSV6764,28,Female,120,157/102,73,1,0,0,1,...,3.833038,217881,23.993866,617,4,9,Canada,North America,Northern Hemisphere,0
8760,XKA5925,47,Male,250,161/75,105,0,1,1,1,...,2.375214,36998,35.406146,527,4,4,Brazil,South America,Southern Hemisphere,1
8761,EPE6801,36,Male,178,119/67,60,1,0,1,0,...,0.029104,209943,27.294020,114,2,8,Brazil,South America,Southern Hemisphere,0


check and drop irrelevant data:

In [112]:
print(data.isnull().sum)

bp = data['Blood Pressure'].str.split('/', expand=True)
data['BP_Systolic']  = pd.to_numeric(bp[0], errors='coerce')
data['BP_Diastolic'] = pd.to_numeric(bp[1], errors='coerce')
data["Sex"] = data["Sex"].map({'Male':1,'Female':0}) #1 represents male, 0 represents female.
data["Diet"] = data["Diet"].map({'Unhealthy':-1,'Average':0, 'Healthy':1}) #-1 represents unhealthy, 0 represents average, 1 re[resents healthy.
data = data.drop(columns=["Blood Pressure", "Hemisphere", "Patient ID", "Continent", "Country"])
data



<bound method DataFrame.sum of       Patient ID    Age    Sex  Cholesterol  Blood Pressure  Heart Rate  \
0          False  False  False        False           False       False   
1          False  False  False        False           False       False   
2          False  False  False        False           False       False   
3          False  False  False        False           False       False   
4          False  False  False        False           False       False   
...          ...    ...    ...          ...             ...         ...   
8758       False  False  False        False           False       False   
8759       False  False  False        False           False       False   
8760       False  False  False        False           False       False   
8761       False  False  False        False           False       False   
8762       False  False  False        False           False       False   

      Diabetes  Family History  Smoking  Obesity  ...  \
0        Fa

,Age,Sex,Cholesterol,Heart Rate,Diabetes,Family History,Smoking,Obesity,Alcohol Consumption,Exercise Hours Per Week,...,Stress Level,Sedentary Hours Per Day,Income,BMI,Triglycerides,Physical Activity Days Per Week,Sleep Hours Per Day,Heart Attack Risk,BP_Systolic,BP_Diastolic
0,67,1,208,72,0,0,1,0,0,4.168189,...,9,6.615001,261404,31.251233,286,0,6,0,158,88
1,21,1,389,98,1,1,1,1,1,1.813242,...,1,4.963459,285768,27.194973,235,1,7,0,165,93
2,21,0,324,72,1,0,0,0,0,2.078353,...,9,9.463426,235282,28.176571,587,4,4,0,174,99
3,84,1,383,73,1,1,1,0,1,9.828130,...,9,7.648981,125640,36.464704,378,3,4,0,163,100
4,66,1,318,93,1,1,1,1,0,5.804299,...,6,1.514821,160555,21.809144,231,1,5,0,91,88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8758,60,1,121,61,1,1,1,0,1,7.917342,...,8,10.806373,235420,19.655895,67,7,7,0,94,76
8759,28,0,120,73,1,0,0,1,0,16.558426,...,8,3.833038,217881,23.993866,617,4,9,0,157,102
8760,47,1,250,105,0,1,1,1,1,3.148438,...,5,2.375214,36998,35.406146,527,4,4,1,161,75
8761,36,1,178,60,1,0,1,0,0,3.789950,...,5,0.029104,209943,27.294020,114,2,8,0,119,67


In [113]:
# data = (data - data.mean())/data.std()
# for column in data.columns:
#     out_of_range = (data[column] < -3) | (data[column] > 3)
#     if out_of_range.any():
#         print(f"Column '{column}' has values outside [-3, 3]")
#     else:
#         print(f"✅ Column '{column}' is all within [-3, 3]")

In [114]:
valid_ranges = {
    'Age': (0, 120),
    'Cholesterol': (100, 500),
    'Systolic BP': (70, 250),
    'Diastolic BP': (40, 150),
    'Heart Rate': (30, 200),
    'Stress Level': (0, 10),
    'Sedentary Hours Per Day': (0, 24),
    'Income': (0, 1_000_000),
    'BMI': (10, 70),
    'Triglycerides': (30, 1000),
    'Exercise Hours Per Week': (0, 40),
    'Physical Activity Days Per Week': (0, 7),
    'Sleep Hours Per Day': (0, 24),
}
print("🔎 Checking for invalid values in your dataset...")
for col, (min_val, max_val) in valid_ranges.items():
    if col in data.columns:
        invalid = data[(data[col] < min_val) | (data[col] > max_val)]
        if not invalid.empty:
            print(f"\n⚠️ Column '{col}' has {len(invalid)} invalid value(s) outside range ({min_val}, {max_val}):")
            print(invalid[[col]])

# Optional summary count
print("\n📊 Summary of invalid values:")
for col, (min_val, max_val) in valid_ranges.items():
    if col in data.columns:
        count = ((data[col] < min_val) | (data[col] > max_val)).sum()
        print(f"{col}: {count} invalid value(s)")

🔎 Checking for invalid values in your dataset...

📊 Summary of invalid values:
Age: 0 invalid value(s)
Cholesterol: 0 invalid value(s)
Heart Rate: 0 invalid value(s)
Stress Level: 0 invalid value(s)
Sedentary Hours Per Day: 0 invalid value(s)
Income: 0 invalid value(s)
BMI: 0 invalid value(s)
Triglycerides: 0 invalid value(s)
Exercise Hours Per Week: 0 invalid value(s)
Physical Activity Days Per Week: 0 invalid value(s)
Sleep Hours Per Day: 0 invalid value(s)


part 1 - Decision trees

In [115]:
# X = data.drop("Heart Attack Risk", axis=1)
# y = data["Heart Attack Risk"]
#
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# # Initialize and train the classifier
# model = DecisionTreeClassifier(random_state=42)
# model.fit(X_train, y_train)

In [116]:
# Predict on the test set
# y_pred = model.predict(X_test)
#
# # Print accuracy
# print("Accuracy:", accuracy_score(y_test, y_pred))
#
# # Detailed report
# print(classification_report(y_test, y_pred))
#
# from sklearn import tree
# import matplotlib.pyplot as plt
#
# clf = DecisionTreeClassifier(max_depth=3)
# clf.fit(X, y)
#
# plt.figure(figsize=(15, 8))
# tree.plot_tree(clf, feature_names=X.columns, class_names=["No Risk", "Risk"], filled=True)
# plt.show()

now, we want to build implementation of decision tree:

but, this implementation is not good enough (long runtime). so instead of using lists, we use numpy functions for better efficiency. on this version, we dont write the Question class, and we customize the functions (especially find best split)

In [117]:
class DecisionTree:
    def __init__(self, max_depth=10, min_samples_leaf=1, features=None):
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.features = features
        self.root = None

    class Leaf:
        def __init__(self, labels):
            self.predictions = Counter(labels)

    class DecisionNode:
        def __init__(self, feature, value, true_branch, false_branch):
            self.feature = feature
            self.value = value
            self.true_branch = true_branch
            self.false_branch = false_branch

    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)
        self.root = self._build_tree(X, y, depth=0)

    def predict(self, X):
        return [self._predict(row, self.root) for row in np.array(X)]

    def _build_tree(self, X, y, depth):
        if len(set(y)) == 1 or depth >= self.max_depth or len(X) < self.min_samples_leaf:
            return self.Leaf(y)

        gain, feature, value, left_idx, right_idx = self._find_best_split(X, y)

        if gain == 0 or len(left_idx) == 0 or len(right_idx) == 0:
            return self.Leaf(y)

        left_branch = self._build_tree(X[left_idx], y[left_idx], depth + 1)
        right_branch = self._build_tree(X[right_idx], y[right_idx], depth + 1)

        return self.DecisionNode(feature, value, left_branch, right_branch)

    def _gini(self, labels):
        mapped = (labels + 1) // 2  # map {-1, 1} → {0, 1}
        counts = np.bincount(mapped)
        probs = counts / len(labels)
        return 1 - np.sum(probs ** 2)

    def _info_gain(self, left_y, right_y, current_uncertainty):
        p = len(left_y) / (len(left_y) + len(right_y))
        return current_uncertainty - p * self._gini(left_y) - (1 - p) * self._gini(right_y)

    def _find_best_split(self, X, y):
        best_gain = 0
        best_feature = None
        best_value = None
        best_left_idx = None
        best_right_idx = None
        current_uncertainty = self._gini(y)

        n_features = self.features if self.features is not None else range(X.shape[1])

        for feature in n_features:
            values = np.unique(X[:, feature])
            for val in values:
                left_idx = np.where(X[:, feature] >= val)[0]
                right_idx = np.where(X[:, feature] < val)[0]

                if len(left_idx) == 0 or len(right_idx) == 0:
                    continue

                gain = self._info_gain(y[left_idx], y[right_idx], current_uncertainty)

                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_value = val
                    best_left_idx = left_idx
                    best_right_idx = right_idx

        return best_gain, best_feature, best_value, best_left_idx, best_right_idx

    def _predict(self, row, node):
        if isinstance(node, self.Leaf):
            return node.predictions.most_common(1)[0][0]

        if row[node.feature] >= node.value:
            return self._predict(row, node.true_branch)
        else:
            return self._predict(row, node.false_branch)

# Prepare NumPy arrays for training
X = data.drop(columns=["Heart Attack Risk"]).to_numpy()
y = data["Heart Attack Risk"].to_numpy()

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the optimized decision tree
fast_tree = DecisionTree(max_depth=5)
fast_tree.fit(X_train, y_train)

# Predict and compute accuracy
y_pred = fast_tree.predict(X_test)
fast_accuracy = np.mean(np.array(y_pred) == y_test)
fast_accuracy

    # # Tree Visualization Methods
    # def print_tree(self, node=None, spacing="", depth=0, max_depth=5):
    #     """Print the tree structure in a readable format"""
    #     if node is None:
    #         node = self.root
    #
    #     if depth > max_depth:
    #         print(spacing + "... (tree continues)")
    #         return
    #
    #     if isinstance(node, Leaf):
    #         total_samples = sum(node.predictions.values())
    #         most_common = node.predictions.most_common(1)[0]
    #         print(f"{spacing}Predict: {most_common[0]} (samples: {total_samples}, confidence: {most_common[1]/total_samples:.2f})")
    #         return
    #
    #     # Print the question
    #     feature_name = self.feature_names[node.question.feature] if self.feature_names else f"feature_{node.question.feature}"
    #     print(f"{spacing}{feature_name} >= {node.question.value:.2f}?")
    #
    #     # Print the branches
    #     print(f"{spacing}--> True:")
    #     self.print_tree(node.true_branch, spacing + "  ", depth + 1, max_depth)
    #
    #     print(f"{spacing}--> False:")
    #    self.print_tree(node.false_branch, spacing + "  ", depth + 1, max_depth)


    # def visualize_graphviz(self, node=None, dot=None, node_id=0):
    #     """Visualize the decision tree using Graphviz"""
    #     if dot is None:
    #         dot = Digraph(comment="Decision Tree")
    #         node = self.root
    #
    #     current_id = str(node_id)
    #
    #     if isinstance(node, Leaf):
    #         prediction = node.predictions.most_common(1)[0]
    #         total = sum(node.predictions.values())
    #         label = f"Predict: {prediction[0]}\nSamples: {total}\nConfidence: {prediction[1] / total:.2f}"
    #         dot.node(current_id, label, shape='box', style='filled', fillcolor='lightblue')
    #         return dot, node_id
    #
    #     feature_name = self.feature_names[node.question.feature] if hasattr(self, "feature_names") and self.feature_names else f"feature_{node.question.feature}"
    #     question_label = f"{feature_name} >= {node.question.value:.2f}"
    #     dot.node(current_id, question_label, shape='ellipse')
    #
    #     # True branch
    #     true_id = node_id + 1
    #     dot.edge(current_id, str(true_id), label="True")
    #     dot, next_id = self.visualize_graphviz(node.true_branch, dot, true_id)
    #
    #     # False branch
    #     false_id = next_id + 1
    #     dot.edge(current_id, str(false_id), label="False")
    #     dot, next_id = self.visualize_graphviz(node.false_branch, dot, false_id)
    #
    #     return dot, next_id


0.638904734740445

In [118]:
# tree.feature_names = feature_names  # This is already known
# dot, _ = tree.visualize_graphviz()
# dot.render("tree", format="png", cleanup=True)  # Generates tree.png

# Image(filename="tree.png")


now, we will implement Random Forest, that will use in our Decision Tree.

In [119]:

class RandomForest:
    def __init__(self, n_trees=10, max_depth=10, min_samples_leaf=1, max_features=None):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.trees = []

    def select_features(self, n_features):
        if self.max_features is None:
            return random.sample(range(n_features), int(np.sqrt(n_features)))
        elif isinstance(self.max_features, int):
            return random.sample(range(n_features), self.max_features)
        elif isinstance(self.max_features, float):
            return random.sample(range(n_features), int(self.max_features * n_features))
        else:
            raise ValueError("Invalid max_features value")

    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)
        n_samples, n_features = X.shape

        self.trees = []
        for _ in range(self.n_trees):
            # Bootstrap sample
            indices = np.random.choice(n_samples, size=n_samples, replace=True)
            X_sample = X[indices]
            y_sample = y[indices]

            # Feature subset
            feature_subset = list(self.select_features(n_features))

            # Train decision tree
            tree = DecisionTree(
                max_depth=self.max_depth,
                min_samples_leaf=self.min_samples_leaf,
                features=list(range(len(feature_subset)))  # local indices for the tree
                )

            # Select only the columns in the feature subset
            X_selected = X_sample[:, feature_subset]
            tree.fit(X_selected, y_sample)
            tree.feature_subset = feature_subset  # Store for use in prediction

            self.trees.append(tree)

    def predict(self, X):
        X = np.array(X)
        tree_preds = []

        for tree in self.trees:
            X_selected = X[:, tree.feature_subset]
            preds = tree.predict(X_selected)
            tree_preds.append(preds)

        # Majority vote
        tree_preds = np.array(tree_preds).T  # shape (n_samples, n_trees)
        final_preds = [Counter(row).most_common(1)[0][0] for row in tree_preds]
        return final_preds

# Train Random Forest
forest = RandomForest(n_trees=10, max_depth=7, max_features=7)
forest.fit(X_train, y_train)

# Predict
y_pred_rf = forest.predict(X_test)
rf_accuracy = np.mean(y_pred_rf == y_test)
print("Random Forest Accuracy:", rf_accuracy)


Random Forest Accuracy: 0.6417569880205363


now, we will implement Adaboost algorithm:

In [ ]:

class Adaboost:
    def __init__(self, n_estimators=20):
        self.n_estimators = n_estimators
        self.learners = []
        self.alphas = []

    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)

        # Convert y from {0, 1} to {-1, 1} internally
        y_signed = np.array([1 if label == 1 else -1 for label in y])
        n = len(y)
        weights = np.ones(n) / n
        self.learners = []
        self.alphas = []

        for i in range(self.n_estimators):
            # Bootstrap sample using weights
            indices = np.random.choice(n, size=n, replace=True, p=weights)
            X_sample = X[indices]
            y_sample = y_signed[indices]

            # Train weak learner (decision stump)
            stump = DecisionTree(max_depth=2)
            stump.fit(X_sample, y_sample)

            # Predict on full training set
            y_pred = stump.predict(X)
            y_pred = np.array([1 if p == 1 else -1 for p in y_pred])  # ensure {-1, 1}

            # Compute weighted error
            error = np.sum(weights[y_pred != y_signed])
            error = max(error, 1e-10)  # avoid divide by zero

            if error > 0.5:
                continue  # skip this stump

            alpha = 0.5 * np.log((1 - error) / error)

            # Update weights
            weights *= np.exp(-alpha * y_signed * y_pred)
            weights /= np.sum(weights)  # normalize

            self.learners.append(stump)
            self.alphas.append(alpha)

    def predict(self, X):
        X = np.array(X)
        scores = np.zeros(len(X))

        for stump, alpha in zip(self.learners, self.alphas):
            pred = stump.predict(X)
            pred = np.array([1 if p == 1 else -1 for p in pred])
            scores += alpha * pred

        # Convert final sign score back to {0, 1}
        return [1 if s > 0 else 0 for s in scores]

    def predict_proba(self, X):
        X = np.array(X)
        scores = np.zeros(len(X))

        for stump, alpha in zip(self.learners, self.alphas):
            pred = stump.predict(X)
            pred = np.array([1 if p == 1 else -1 for p in pred])
            scores += alpha * pred

        # Use sigmoid to normalize to [0, 1]
        probs = 1 / (1 + np.exp(-2 * scores))
        return [{'0': 1 - p, '1': p} for p in probs]

# y in {0, 1}
X = data.drop(columns=["Heart Attack Risk"]).to_numpy()
y = data["Heart Attack Risk"].to_numpy()

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Adaboost
model = Adaboost(n_estimators=20)
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)
accuracy = np.mean(np.array(y_pred) == y_test)
print("Adaboost Accuracy:", accuracy)

Adaboost Accuracy: 0.6332002281802624
